In [6]:
# ------- ONE-TIME HOT-RELOAD (run after editing files) -------------
import importlib

# reload service with new 3-arg resolve_predicate
import deps.edge_type_service as ets
importlib.reload(ets)

# reload extractor so it re-imports the updated service
import extractors.edge_extractor as ee
importlib.reload(ee)

# ------- CONFIG ----------------------------------------------------
ee.EXPECTED_ABSTRACTS[:] = ["located_in", "founded_by", "acquired_by"]

sentences = [
    "Zest Airways was founded by Alfredo Yao in Manila.",
    "Google acquired YouTube for $1.65 billion in 2006.",
    "AirAsia Zest is located in Manila."
]

# ------- EXTRACTION + LIVE TRACE -----------------------------------
edges = [edge for s in sentences if (edge := ee.extract_edge(s, verbose=True))]

print(f"\n{len(edges)} edges extracted ✔️")
print("Edge counts:", dict(ee.EDGE_COUNTS))
print("Alias-tier counts:", dict(ee.ALIAS_TIER_COUNTS))



Passive→active rewrite
Alias tier=-1
Fine predicate: founded_by
▶ resolving 'founded_by'
Dict-A hit (seed) → founded_by
Abstract type (Dict-A) = founded_by
-- Edge accepted
Alias tier=-1
Fine predicate: acquired_by
▶ resolving 'acquired_by'
Dict-A hit (seed) → acquired_by
Abstract type (Dict-A) = acquired_by
-- Edge accepted
Alias tier=-1
Fine predicate: located_in
▶ resolving 'located_in'
Dict-A hit (seed) → located_in
Abstract type (Dict-A) = located_in
-- Edge accepted

3 edges extracted ✔️
Edge counts: {'founded_by': 1, 'acquired_by': 1, 'located_in': 1}
Alias-tier counts: {}


In [3]:
import importlib, extractors.triple_extractor as te
importlib.reload(te)
import importlib, extractors.edge_extractor as ee
importlib.reload(ee)

from deps.graph_store import GraphStore
import extractors.edge_extractor as ee

ee.EXPECTED_ABSTRACTS[:] = ["located_in", "founded_by", "acquired_by"]

doc_id = "docA"
sentences = [
    "Zest Airways, founded by Alfredo Yao, is located in Manila and was acquired by AirAsia.",
    "In 2006 Google acquired YouTube for $1.65 billion."
]

store = GraphStore()
for i, s in enumerate(sentences):
    store.add_sentence(doc_id, i, s,
                       extract_fn=ee.extract_sentence_graph,
                       verbose=True)

print("Doc-level graph has", len(store.get_doc_graph(doc_id)), "edges")



Sentence 'Zest Airways, founded by Alfredo Yao, is located in Manila and was acquired by AirAsia.'
Passive→active rewrite
Alias tier=-1
-- produced 0 edge(s)


Sentence 'In 2006 Google acquired YouTube for $1.65 billion.'
Alias tier=-1
-- produced 0 edge(s)

Doc-level graph has 0 edges


In [7]:
# ------------- Cell 2 -------------
import pandas as pd, numpy as np
df = pd.DataFrame([{
        "edge_type": e["edge_type"],
        "fine_pred": e["meta"]["fine_pred"],
        "eid":       e["meta"]["eid"],
        "tier":      e["meta"]["alias_tier"],
        "cos":       float(e["surface"] @ e["semantic"]) /
                     (np.linalg.norm(e["surface"])*np.linalg.norm(e["semantic"]) + 1e-8)
    } for e in edges])
df


,edge_type,fine_pred,eid,tier,cos
0,founded_by,founded_by,None,-1,0.011447
1,acquired_by,acquired_by,None,-1,0.007623
2,located_in,located_in,None,-1,0.012107
